In [ ]:
# ==========================================================
# NUEVA NOTEBOOK — CELDA ÚNICA: Cargar resultados.zip y predecir un ZIP nuevo
# Entradas (subidas manualmente a /content):
#   - /content/resultados.zip   (bundle del modelo + metadata)
#   - /content/nuevo.zip        (nuevo dataset para inferencia)
# Salidas:
#   - /content/predicciones.csv
# ==========================================================
import os, glob, json, zipfile, shutil, time
import numpy as np
import tensorflow as tf

# -------------------------
# 0) RUTAS
# -------------------------
RESULTS_ZIP = "/content/resultados.zip"
NEW_ZIP     = "/content/simpson_nuevos.zip"  # cambia el nombre si llega con otro nombre

RESULTS_DIR = "/content/resultados"
NEW_WORKDIR = "/content/new_data"

EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
AUTOTUNE = tf.data.AUTOTUNE

assert os.path.isfile(RESULTS_ZIP), "Falta /content/resultados.zip (súbelo manualmente)"
assert os.path.isfile(NEW_ZIP),     "Falta /content/nuevo.zip (súbelo manualmente)"

# -------------------------
# 1) DESCOMPRIMIR resultados.zip
# -------------------------
if os.path.isdir(RESULTS_DIR):
    shutil.rmtree(RESULTS_DIR)
os.makedirs(RESULTS_DIR, exist_ok=True)

with zipfile.ZipFile(RESULTS_ZIP, "r") as z:
    z.extractall(RESULTS_DIR)

print("✅ resultados.zip extraído en:", RESULTS_DIR)
print("Contenido:", sorted(os.listdir(RESULTS_DIR))[:20])

MODEL_PATH = os.path.join(RESULTS_DIR, "model.keras")
META_PATH  = os.path.join(RESULTS_DIR, "metadata.json")

assert os.path.isfile(MODEL_PATH), f"No existe {MODEL_PATH}"
assert os.path.isfile(META_PATH),  f"No existe {META_PATH}"

# -------------------------
# 2) CARGAR METADATA + MODELO
# -------------------------
with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

IMG_SIZE  = tuple(meta["img_size"])
CHANNELS  = int(meta["channels"])
CLASSES   = list(meta["classes"])
BATCH     = int(meta.get("batch_final", 32))

print("\nCONFIG INFERENCIA:")
print("  IMG_SIZE :", IMG_SIZE)
print("  CHANNELS :", CHANNELS)
print("  BATCH    :", BATCH)
print("  #CLASSES :", len(CLASSES))

model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ Modelo cargado:", MODEL_PATH)

# -------------------------
# 3) DESCOMPRIMIR NUEVO ZIP
# -------------------------
if os.path.isdir(NEW_WORKDIR):
    shutil.rmtree(NEW_WORKDIR)
os.makedirs(NEW_WORKDIR, exist_ok=True)

with zipfile.ZipFile(NEW_ZIP, "r") as z:
    z.extractall(NEW_WORKDIR)

print("\n✅ nuevo.zip extraído en:", NEW_WORKDIR)

# -------------------------
# 4) ENCONTRAR TODAS LAS IMÁGENES (sin asumir carpetas por clase)
#    (Soporta: imágenes en raíz, o en subcarpetas a cualquier profundidad)
# -------------------------
def list_all_images_recursive(root_dir, exts=EXTS):
    files = []
    for r, _, fs in os.walk(root_dir):
        for fn in fs:
            if fn.lower().endswith(exts):
                files.append(os.path.join(r, fn))
    return sorted(files)

files = list_all_images_recursive(NEW_WORKDIR)
if len(files) == 0:
    raise ValueError(f"No encontré imágenes dentro de {NEW_ZIP}")

print("Imágenes encontradas:", len(files))
print("Ejemplo:", files[0])

# -------------------------
# 5) PIPELINE TF.DATA (mismo preprocesamiento que entrenaste)
# -------------------------
def decode_image(path, img_size, channels):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.resize(img, img_size, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds(paths, batch):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(lambda p: decode_image(p, IMG_SIZE, CHANNELS), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

ds = make_ds(files, BATCH)

# -------------------------
# 6) PREDICCIÓN + CSV
# -------------------------
probs = model.predict(ds, verbose=0)
pred_idx = np.argmax(probs, axis=1)
pred_cls = [CLASSES[i] for i in pred_idx]
conf = np.max(probs, axis=1)

out_csv = "/content/predicciones.csv"
import csv
with open(out_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["filepath", "pred_idx", "pred_class", "confidence"])
    for p, i, c, cf in zip(files, pred_idx, pred_cls, conf):
        w.writerow([p, int(i), c, float(cf)])

print("\n✅ Listo. CSV:", out_csv)
print("Primeras 5 predicciones:")
for k in range(min(5, len(files))):
    print(f"  {os.path.basename(files[k])} -> {pred_cls[k]} (conf={conf[k]:.3f})")
